<h1>Preparation of the German Twitter Data</h1>

<p>This file is for preparing the german tweets. The preparation is mainly for training my own model for sentiment detection with Flair. </p>

In [1]:
import csv
import numpy as np
import pandas as pd
import re
import nltk

<h1>Einlesen und bestimmung der Felder</h1>
<p>Durch die weitern Operatoren, kann ich die einzelnen Felder in DF bestimmen. Durch die Lambda Funktion werden die Str zu Listen, was die spätere verarbeitung erleichtert. 
    Durch die Parse_dates Funktion wird das Datumsfeld zu einem Timestamp, was die weiter Verarbeitung wesentlich leichter macht.</p>

In [23]:
df_manager_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_DE/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_manager_de = df_manager_de.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)

In [257]:
df_manager_de.head()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
0,703311366262800384,1392131804,2016-02-26 20:11:11+00:00,703311366262800384,de,0,0,2,0,Twitter Web Client,None,None,#Wirecard-Chef Markus Braun pumpt irrwitzige S...
1,702405572012974080,1053723204,2016-02-24 08:11:53+00:00,702405572012974080,de,0,0,0,0,Twitter Web Client,None,None,Jetzt günstig einsammeln und später absahnen? ...
2,701784983829143553,554491983,2016-02-22 15:05:53+00:00,701784983829143553,de,0,0,0,0,Facebook,None,None,Morgen startet die neue Reihe R²-JazzCube! Vol...
3,701446667170615296,3365669884,2016-02-21 16:41:32+00:00,701446667170615296,de,0,0,1,0,Twitter Web Client,None,None,die Y-Achse des #prusai3 nimmt Formen an :-) ...
4,700597812556386304,4858383167,2016-02-19 08:28:29+00:00,700597812556386304,de,2,0,0,0,Twitter for iPhone,retweeted,700578193334411264,RT @aktionaer: #Wirecard: Was führt der Vorsta...


<br>
<p>Prüfen welche Datenfelder in den einzelnen Zeilen enthalten sind. War wichtig um diese anpassen zu können.</p>

In [204]:
print("conversation_id: ", df_manager_de.iloc[0]["conversation_id"], "type: ", type(df_manager_de.iloc[0]["conversation_id"]))
print("author_id: ", df_manager_de.iloc[0]["author_id"], "type: ", type(df_manager_de.iloc[0]["author_id"]))
print("created_at: ", df_manager_de.iloc[0]["created_at"], "type: ", type(df_manager_de.iloc[0]["created_at"]))
print("tweet_id: ", df_manager_de.iloc[0]["tweet_id"], "type: ", type(df_manager_de.iloc[0]["tweet_id"]))
print("lang: ", df_manager_de.iloc[0]["lang"], "type: ", type(df_manager_de.iloc[0]["lang"]))
print("retweet_count: ", df_manager_de.iloc[0]["retweet_count"], "type: ", type(df_manager_de.iloc[0]["retweet_count"]))
print("reply_count: ", df_manager_de.iloc[0]["reply_count"], "type: ", type(df_manager_de.iloc[0]["reply_count"]))
print("like_count: ", df_manager_de.iloc[0]["like_count"], "type: ", type(df_manager_de.iloc[0]["like_count"]))
print("quote_count: ", df_manager_de.iloc[0]["quote_count"], "type: ", type(df_manager_de.iloc[0]["quote_count"]))
print("source: ", df_manager_de.iloc[0]["source"], "type: ", type(df_manager_de.iloc[0]["source"]))
print("tweet_type: ", df_manager_de.iloc[0]["tweet_type"], "type: ", type(df_manager_de.iloc[0]["tweet_type"]))
print("referenced_tweet_id: ", df_manager_de.iloc[0]["referenced_tweet_id"], "type: ", type(df_manager_de.iloc[0]["referenced_tweet_id"]))
print("text: ", df_manager_de.iloc[0]["text"], "type: ", type(df_manager_de.iloc[0]["text"]))

conversation_id:  703311366262800384 type:  <class 'numpy.int64'>
author_id:  1392131804 type:  <class 'numpy.int64'>
created_at:  2016-02-26 20:11:11+00:00 type:  <class 'pandas._libs.tslibs.timestamps.Timestamp'>
tweet_id:  703311366262800384 type:  <class 'numpy.int64'>
lang:  de type:  <class 'str'>
retweet_count:  0 type:  <class 'numpy.int64'>
reply_count:  0 type:  <class 'numpy.int64'>
like_count:  2 type:  <class 'numpy.int64'>
quote_count:  0 type:  <class 'numpy.int64'>
source:  Twitter Web Client type:  <class 'str'>
tweet_type:  None type:  <class 'str'>
referenced_tweet_id:  None type:  <class 'str'>
text:  #Wirecard-Chef Markus Braun pumpt irrwitzige Summen in sein Unternehmen. Woher stammt das Geld? @SchreiberDohms  https://t.co/VdsV4nvBe4 type:  <class 'str'>


<h3>Löschen von unnötigen Informationen</h3>
<p>Hier werden Informationen aus den Tweets Texten, die nicht weiter benötigt werden. Bspw. Links</p>

In [24]:
for each in range(len(df_manager_de)):
    df_manager_de.loc[df_manager_de["text"] == 
                      df_manager_de.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('#', '', 
                                                                                       df_manager_de.loc[each]["text"]))))

In [25]:
df_manager_de.iloc[0]["text"]

'Wirecard-Chef Markus Braun pumpt irrwitzige Summen in sein Unternehmen. Woher stammt das Geld?   '

<br>
<h3>Nicht Kontextbezogene Tweets löschen</h3>
<p>So kann ich prüfen ob bestimmte Begriffe in den Texten enthalten sind und mir die conversation_id ausgeben lassen. Diese lasse ich dann löschen und auch zugehörige Tweets, da diese nicht in den Kontext passen.<br>Die wörter in der Liste müssen klein geschrieben werden, da auch alle Buchstaben für die Prüfung klein geschrieben werden müssen, da sinst nicht alles erkannt wird.</p>
<p>Zu löschende Begriffe</p>
<ul>
    <li>BVB</li>
    <li>Fußball</li>
    <li>Fussball</li>
    <li>Dortmund</li>
</ul>

In [26]:
not_used_words_list = ["bvb", "fußball", "fussball", "dortmund"]
tweet_id_without_context_list = []
for each in range(len(df_manager_de)):
    for word in not_used_words_list:
        if word in df_manager_de.iloc[each]["text"].lower():
            if df_manager_de.iloc[each]["tweet_id"] not in tweet_id_without_context_list:
                tweet_id_without_context_list.append(df_manager_de.iloc[each]["tweet_id"])

<p>Iteration durch die Liste mit zu löschenden Tweets, bis der Wert zu Beginn und zum Ende gleich ist und es keine
weiteren Tweets mehr gibt, die gefiltert werden müssen. </p>

In [27]:
def check_for_further_tweets():
    start_len = len(tweet_id_without_context_list)
    for each in range(len(tweet_id_without_context_list)):
        df_tweets = df_manager_de.loc[(df_manager_de["conversation_id"] == tweet_id_without_context_list[each]) | 
               (df_manager_de["referenced_tweet_id"] == str(tweet_id_without_context_list[each]))]

        for each in range(len(df_tweets)):
            if df_tweets.iloc[each]["tweet_id"] not in tweet_id_without_context_list:
                tweet_id_without_context_list.append(df_tweets.iloc[each]["tweet_id"])
                
    if start_len != len(tweet_id_without_context_list):
        check_for_further_tweets()
check_for_further_tweets()

<p>Löschen der nicht kontextbezogenen Tweets und den Index neu setzen um später keine Probleme mit dem Index zu bekommen.</p>

In [28]:
for each in tweet_id_without_context_list:
    df_manager_de = df_manager_de.drop(df_manager_de.loc[df_manager_de["tweet_id"] == each].index[0])
df_manager_de = df_manager_de.reset_index(drop=True)

<br>
<h4>Check</h4>
<p>Aufrufen einer Line um zu prüfen, welche Informationen enthalten sind.<br>
    Den Index abrufen um mir nur den Text in voller Länge anzeigen zu lassen, da dieser sonst gekürzt ist.</p>
   

In [69]:
df_manager_de.loc[df_manager_de["conversation_id"] == 1098841597582737409]

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text


In [49]:
df_manager_de.loc[df_manager_de.index[df_manager_de["conversation_id"] == 1278435775441494016].tolist()[0]]["text"]

'RT @RNBVB: 👨🏻\u200d⚕️ Dr. Markus #Braun ist vom Nachrichtenmagazin #Focus als "Top-Mediziner" in Deutschland ausgezeichnet worden. Zuletzt arbei…'

In [48]:
print(df_manager_de.index[df_manager_de["conversation_id"] == 1278435775441494016].tolist()[0])

14209


In [36]:
df_manager_de.loc[14209]["text"]

'RT @RNBVB: 👨🏻\u200d⚕️ Dr. Markus #Braun ist vom Nachrichtenmagazin #Focus als "Top-Mediziner" in Deutschland ausgezeichnet worden. Zuletzt arbei…'

<br>
<h3>Löschen von Retweets</h3>
<p>Alle Retweets löschen, da diese in diesem Schritt nicht benötigt werden.</p>

In [29]:
counter = 0
retweet_list = []
for each in range(len(df_manager_de)):
    if "retweeted" in df_manager_de.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)

5428


In [30]:
df_manager_de = df_manager_de.drop(retweet_list)

<h1>Löschen von doppelten Tweets</h1>
<p>Alle Tweets, die dopplet im Datensatz enthalten sind, anhand der tweet_id werden gelöscht. Somit werden auch die zuvor aufgespalteten Listen wieder gelöscht</p>

In [107]:
df_manager_de = df_manager_de.drop_duplicates(subset=["tweet_id"])

In [108]:
len(df_manager_de)

8836

<h1>Check User</h1>
<p>Überprüfe die User Daten nach Namen, die nichts mit dem Thema Wirecard zu tun haben. Auch Nutzer, die ggf. nur durch Zufall in die Abfrage gekommen sind, z.B. durch den gleichen Namen wie einer der Manager, sollen aus dem Datensatz entnommen werden.<br>Auffällige Namen sind <b>Markus Braun</b> und <b>Jan Marsalek</b>, da es sich um den CEO und den verantwortlichen Manager handelt.</p>

In [31]:
df_user = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_DE/user.csv", sep=";")

<p>Prüfen nach allen Nutzern mit dem Namen Markus</p>

In [32]:
df_user.loc[df_user.name.str.lower().str.contains("markus")].drop_duplicates()

,user_id,username,name,follower,following,tweet_cound,list_count,verified,created_at
3,3365669884,MarkusB1980,Markus Braun,82,181,620,13,False,2015-07-08 08:43:24+00:00
53,170698930,Magix_Markus,Magic Markus,163,192,24324,18,False,2010-07-25 14:30:29+00:00
58,827452635057881088,_MarkusBraun,Markus Braun,31414,88,1162,81,True,2017-02-03 09:44:16+00:00
91,536335486,Advobraun,RA Markus Braun,17,93,16,0,False,2012-03-25 14:09:25+00:00
253,745165314,mvdweyer,Markus van de Weyer,1141,1297,35732,199,False,2012-08-08 13:08:41+00:00
951,1174753444189671424,MarkusW28522797,Markus Wolf,0,4,2,0,False,2019-09-19 18:33:51+00:00
1053,1201441525,MarkusFasse,Markus Fasse,1688,1065,2299,53,False,2013-02-20 15:49:37+00:00
1298,780754694419296256,markuscbraun1,Markus C. Braun,85,232,89,0,False,2016-09-27 13:03:19+00:00
1347,1053554251367174145,MWeingran,MarkusWeingran,176,50,325,3,False,2018-10-20 07:51:22+00:00
1506,855232346,MarkusBlume,Markus Blume,12911,284,1333,196,True,2012-09-30 19:26:22+00:00


<p>Prüfen auf auffäliige Einträge bezüglich Jan Marksalek</p>

In [33]:
df_user.loc[df_user.name.str.lower().str.contains("jan")].drop_duplicates()

,user_id,username,name,follower,following,tweet_cound,list_count,verified,created_at
181,152703288,dejanjovicevic,Dejan Jovicevic,1785,1449,6801,25,False,2010-06-06 17:14:56+00:00
471,960630599601082368,JHendrikP,Jan,61,143,2741,1,False,2018-02-05 21:45:57+00:00
626,571011183,SeraelJann,SeraelJann,5,18,78,0,False,2012-05-04 15:40:39+00:00
791,427754037,clauci2nd,claudia janecek - food & medicine #Yemen 🇾🇪,3504,4269,79378,16,False,2011-12-03 22:56:56+00:00
1089,861221059747610624,proudtobehay,Django,48,803,2398,4,False,2017-05-07 14:07:56+00:00
1120,48455041,Willmroth,Jan Diesteldorf,1995,957,1817,75,False,2009-06-18 19:14:48+00:00
1456,1169762222790385665,Jannis73502821,Jannis,2,95,21,0,False,2019-09-06 00:00:28+00:00
1488,926892715706593280,JanKaiserICH,Jan Kaiser,3,22,2,0,False,2017-11-04 19:23:39+00:00
1915,1009685810,DanielaLongding,daniel janson,10,100,244,0,False,2012-12-13 20:49:01+00:00
1946,1225022850735771648,JMoldau,Jan Moldau,0,10,613,0,False,2020-02-05 11:46:40+00:00


<p>Auffällige User_IDs:</p>
<ul>Markus Braun
    <li>3365669884 - MarkusB1980</li>
    <li>827452635057881088 - _MarkusBraun</li>
    <li>536335486 - Advobraun</li>
    <li>780754694419296256 - markuscbraun1</li>
</ul>
<ul>Jan (Marsalek)
    <li>19072286 - janboehm</li>
</ul>

<p>Anzeigen der Tweets der "auffälligen" Nutzer</p>

In [40]:
pd.set_option('display.max_colwidth', None)
df_manager_de.loc[df_manager_de["author_id"] == 3365669884]["text"]

3        die Y-Achse des  prusai3 nimmt Formen an :-)  3ddrucker 3dprint
10               Der Rahmen vom prusai3 ist zusammen gebaut :-)  3dprint
18    "kleiner" Bericht zum Grillen mit Freunden  grillen bbq pulledpork
Name: text, dtype: object

In [41]:
pd.set_option('display.max_colwidth', None)
df_manager_de.loc[df_manager_de["author_id"] == 827452635057881088]["text"]

Series([], Name: text, dtype: object)

In [42]:
pd.set_option('display.max_colwidth', None)
df_manager_de.loc[df_manager_de["author_id"] == 536335486]["text"]

86    NEUE HOMPAGE MIT NEUEN FUNKTIONEN 
Name: text, dtype: object

In [43]:
pd.set_option('display.max_colwidth', None)
df_manager_de.loc[df_manager_de["author_id"] == 780754694419296256]["text"]

Series([], Name: text, dtype: object)

In [44]:
pd.set_option('display.max_colwidth', None)
df_manager_de.loc[df_manager_de["author_id"] == 19072286]["text"]

Series([], Name: text, dtype: object)

<p>Nur bei zwei der Nutzer ist nach der Vorverarbeitung noch ein Tweet in der Liste. Diese beiden Nutzer tragen nichts zum Thema Wirecard bei und twittern nicht diesbezüglich. <br>Ich habe Jan bei der Suche nach Jan lediglich den Namen Jan Böhmermann geprüft, da es sich hierbei um eine bekannte Persönlichkeit handelt, die zu diesem aber auch zu vielen anderen Themen twittert und ich dies deswegen überprüfen wollte.</p>

In [45]:
df_user["name"].value_counts().head(50)

Markus Braun                                   44
Handelsblatt                                   29
Wirecard                                       27
kein Spekulant blauer Haken                    26
FAZ Finanzen                                   26
Dr. Fred... no MD/PD/XYZ-D                     25
Ehssan Khazaeli                               25
AMCtoooMoon                                    25
HandelsblattFinanzen                           24
manager magazin                                24
csfa.                                          22
A.Barendrecht                                  21
Daniel                                         20
cS-Strategies                                  20
Frankfurter Allgemeine gesamt                  20
Mario                                          20
Nils Wischmeyer                                20
FinanzNachrichten.de                           20
🎲 Policon automatisches Nachrichtenportal 👽    19
WirtschaftsWoche                               19


In [109]:
df_manager_de["author_id"].value_counts().to_frame().reset_index()

,index,author_id
0,1118219555967520769,865
1,1159698800790953984,192
2,3029833842,175
3,2195833356,89
4,1100318139718414336,65
...,...,...
2493,312358120,1
2494,3279547546,1
2495,835150061516558336,1
2496,3009224172,1


In [117]:
df_manager_de["author_id"].value_counts().to_frame().reset_index().join(
    df_user.set_index("user_id"), on="index").drop_duplicates(subset=["index"]).head(11)

,index,author_id,username,name,follower,following,tweet_cound,list_count,verified,created_at
0,1118219555967520769,865,AMCtootheMoon,AMCtoooMoon,274,221,22555,1,False,2019-04-16 18:28:12+00:00
1,1159698800790953984,192,blvddemoulin,A.Barendrecht,159,964,5477,0,False,2019-08-09 05:31:55+00:00
2,3029833842,175,Steinfireflint,Dr. Fred... no MD/PD/XYZ-D,465,323,23065,12,False,2015-02-19 12:35:58+00:00
3,2195833356,89,Ch11110,Ch111,21,149,1422,0,False,2013-11-15 11:36:59+00:00
4,1100318139718414336,65,Tradefair6,Tradefair,44,33,1766,1,False,2019-02-26 08:54:22+00:00
5,704753138146877440,65,csFraudAnalysis,csfa.,1035,561,15064,29,False,2016-03-01 19:40:16+00:00
6,2280206959,59,FreieWeltEu,🎲 Policon automatisches Nachrichtenportal 👽,689,2229,1570308,66,False,2014-01-07 07:14:28+00:00
7,172082646,57,Bonnetmaker,Götz Kluge,268,591,15730,2,False,2010-07-28 21:40:20+00:00
8,1095261591203966976,55,Kaan24263371,Aslan,93,133,4525,1,False,2019-02-12 10:01:27+00:00
9,583334012,52,blockhero_,blockhero.eth,549,1480,15540,2,False,2012-05-18 01:19:29+00:00


In [135]:
pd.set_option('display.max_colwidth', None)
df_manager_de.loc[df_manager_de["author_id"] == 60673185]["text"]

87                                                                                                            Wirecard Chef Markus Braun rechnet mit einer „Vervielfachung“ des Unternehmenswerts. Bin auch voll von dem Unternehmen überzeugt. nobullshit 
288                                                                                                                                                                            Markus Braun kündigt gegenüber  an:  wird boon. zu Smartphone-Bank ausbauen.
389                                                                                    im Interview mit  "Unsere interne Untersuchung ist weitestgehend abgeschlossen. Deshalb kann ich heute schon sagen, dass an den Vorwürfen nichts dran ist.“ Wirecard
1440                                                                                                                                                                                Aschheim zieht Konsequenzen: Wirecard beruft Sonderprüfer.  for 

<p><br><b>Check nach besonderen Namen, die in der Liste besonder oft auftauchen oder Namen, denen eine besondere Bedeutung zugeordnet wird.</b></p>

In [174]:
df_user.loc[df_user.name.str.contains("markus")].drop_duplicates()

,user_id,username,name,follower,following,tweet_cound,list_count,verified,created_at
4888,443638494,markuskrainer1,markus krainer,2,34,10,0,False,2011-12-22 11:25:26+00:00
4953,46415147,kubant,markus,64,340,2786,0,False,2009-06-11 15:46:37+00:00


<p><b>Tweets von den zu prüfenden Leuten ansehen</b></p>

In [194]:
user_id_list = []
for each in df_user.loc[df_user.name.str.lower().str.contains("jan")].drop_duplicates()["user_id"]:
    user_id_list.append(each)
df_manager_de.loc[df_manager_de.author_id.isin(user_id_list)]

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
196,1039392642847535104,152703288,2018-09-11 05:58:11+00:00,1039392642847535104,de,0,0,1,0,LinkedIn,None,None,Speaker Coup: Knapp nach DAX-Listing: Wirecard...
197,1039176827049332736,152703288,2018-09-10 15:40:37+00:00,1039176827049332736,de,2,0,0,0,Twitter for iPhone,retweeted,1039065678353973248,RT @DarwinsCircle: Knapp nach DAX-Listing: Wir...
602,1110837535423504384,960630599601082368,2019-03-27 10:11:37+00:00,1110846829795729408,de,0,1,1,0,Twitter for iPhone,replied_to,1110840122201784320,@jens_algebra @_MarkusBraun Das ist doch jetzt...
1039,1116041090677784578,571011183,2019-04-10 20:15:23+00:00,1116072202288467968,de,0,1,1,0,Twitter Web App,replied_to,1116065141014052864,@AuchDuMeinSohn @Alexand93433205 @Zwitscherpau...
1058,1114423943761473536,571011183,2019-04-06 10:05:16+00:00,1114469109914374144,de,0,0,0,0,Twitter Web App,replied_to,1114467448647028736,@AuchDuMeinSohn @bald_saitama @aveirodobraga @...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14034,1279469432948088832,1261872733,2020-07-04 17:37:38+00:00,1279469432948088832,de,1,0,0,0,Hootsuite Inc.,retweeted,1279467577413193728,RT @welt: Gefälschte Einreiseunterlagen auf de...
14279,1278761776377663489,804934696555704320,2020-07-02 18:45:40+00:00,1278761776377663489,de,19,0,0,0,Twitter for iPhone,retweeted,1278757395892703235,RT @SZ: Die weltweite Suche nach Ex-Wirecard-V...
14291,1278749709469913091,972233251,2020-07-02 17:57:43+00:00,1278749709469913091,de,1,0,0,0,Twitter Web App,retweeted,1278748251315937280,RT @VinzentLeitgeb: Eine Donald-Trump-Pappfigu...
14305,1278732404497223681,48455041,2020-07-02 16:48:57+00:00,1278732404497223681,de,3,0,0,0,Twitter for Android,retweeted,1278729605696884738,"RT @wischmeyer_n: Jan Marsalek, Ex-Vorstand vo..."


In [14]:
df_user.loc[df_user.name.str.lower().str.contains("markus")].drop_duplicates()

,user_id,username,name,follower,following,tweet_cound,list_count,verified,created_at
3,3365669884,MarkusB1980,Markus Braun,82,181,620,13,False,2015-07-08 08:43:24+00:00
53,170698930,Magix_Markus,Magic Markus,163,192,24324,18,False,2010-07-25 14:30:29+00:00
58,827452635057881088,_MarkusBraun,Markus Braun,31414,88,1162,81,True,2017-02-03 09:44:16+00:00
91,536335486,Advobraun,RA Markus Braun,17,93,16,0,False,2012-03-25 14:09:25+00:00
253,745165314,mvdweyer,Markus van de Weyer,1141,1297,35732,199,False,2012-08-08 13:08:41+00:00
951,1174753444189671424,MarkusW28522797,Markus Wolf,0,4,2,0,False,2019-09-19 18:33:51+00:00
1053,1201441525,MarkusFasse,Markus Fasse,1688,1065,2299,53,False,2013-02-20 15:49:37+00:00
1298,780754694419296256,markuscbraun1,Markus C. Braun,85,232,89,0,False,2016-09-27 13:03:19+00:00
1347,1053554251367174145,MWeingran,MarkusWeingran,176,50,325,3,False,2018-10-20 07:51:22+00:00
1506,855232346,MarkusBlume,Markus Blume,12911,284,1333,196,True,2012-09-30 19:26:22+00:00
